# Calorimetry with DNNs

The data from highly granular calorimeters can be viewed as 3D images, making them ideal for image classification problems. In this lab, we will apply image classification to simulated calorimeter data from the LCD detector concept for the CLIC accelerator. We will use the [CaloDNN](https://github.com/UTA-HEP-Computing/CaloDNN) package to systematically study different neural network architectures, optimizers, loss functions, and other hyperparameters.

The data is compused of 4 particle types: electrons, neutral pions (pi0s), charged pions, and photons (gamma). The LCD calorimeter is composed of electromagnetic (ECAL) and hadronic (HCAL) sections. The simulation shoots a single particle into the calorimeter and stores a 25 by 25 by 25 cell part of the ECAL and 5 by 5 by 60 part of the HCAL around the particle. 

## CaloDNN

CaloDNN is a DLKit based package for studying LCD Calorimetry with DNNs.

The package consists of the following files:

  **CaloDNN/ClassificationExperiment.py**: This is the “experiment” that drives everything.

   **CaloDNN/ClassificationArguments.py**: This is the file where all of the above arguments are defined and parsed. You can add your own options here if need be. Some defaults are defined here.

   **CaloDNN/ClassificationScanConfig.py**: This is the configuration file. The model and experiment parameters are set here. This example is setup to allow hyper-parameter scanning. It also contains the list of input files and maps the files and datasets to classes as well as controls what variables are used in the Neural Network.

   **CaloDNN/Models.py**: This contains the Keras models, wrapped in a DLKit ModelWrapper class.

   **CaloDNN/LCDData.py**: Contains the DLGenerators to read the data.
   
Typically we run these experiments from the shell command prompt (e.g. here getting help):

    python -m CaloDNN.ClassificationExperiment --help
    
But we can also do it in our current Jupyter session as follows:


In [4]:
%run -im CaloDNN.ClassificationExperiment -- --help

usage: ClassificationExperiment.py [-h] [-C CONFIG] [-L LOADMODEL]
                                   [--gpu GPUID] [--cpu] [--NoTrain]
                                   [--NoAnalysis] [--LowMem] [--Test]
                                   [--Recover] [-s HYPERPARAMSET] [--nopremix]
                                   [--preload] [-r RUNNINGTIME] [-p]
                                   [--GracefulExit]

optional arguments:
  -h, --help            show this help message and exit
  -C CONFIG, --config CONFIG
                        Use specified configuration file.
  -L LOADMODEL, --LoadModel LOADMODEL
                        Loads a model from specified directory.
  --gpu GPUID           Use specified GPU.
  --cpu                 Use CPU.
  --NoTrain             Do not run training.
  --NoAnalysis          Do not run analysis.
  --LowMem              Minimize Memory Usage.
  --Test                Run in test mode (reduced examples and epochs).
  --Recover             Train only if fail 

The experiment has 4 steps:

    1. Setup Loading Data
    2. Load or Build Model
    3. Train Model
    4. Run Analysis 

You can turn off steps as needed using the flags above. 

You can run a quick test (reduced number of events and epochs) as follows:

In [ ]:
%run -im CaloDNN.ClassificationExperiment -- --Test

Using GPU 0
HyperParameter Scan:  240 possible combiniations.
______________________________________
ScanConfiguration
______________________________________
Picked combination:  0
Combo[0]={'Width': 32, 'Depth': 1, 'lr': 0.01, 'optimizer': "'RMSprop'", 'decay': 0.01}
Model Filename:  CaloDNN_32_1_0.01_RMSprop_0.01
______________________________________
Test Mode: Set MaxEvents to 20000 and Epochs to 10
Using PremixGenerator.
Looking for Previous Model to load.
Loading Previous Model From: TrainedModels.Test/CaloDNN_32_1_0.01_RMSprop_0.01_Merged
Output Directory: TrainedModels.Test/CaloDNN_32_1_0.01_RMSprop_0.01_Merged.1
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 25, 25, 25)    0                                            
____________________________________________________________________

### Configuration and Hyperparameter Scanning

Our DNN architecture is defined by the types, number, and dimension of layers. Hyper-parameter
scanning refers to the process of searching for an optimal architecture that performs well for a
task and can be trained and applied within reasonable time. Beyond the parameters that define the
DNN architecture, other configuration parameters allow setting and testing activation and cost
functions, optimization (e.g. minimization) techniques, and rate other training parameters.

In DLKit, these parameters are set in a configuration file, which defines a single python key/value
dictionary called **Config**. DLKit puts the contents of this dictionary in the global scope with the
keys as the variable names. As an example, see `CaloDNN/ClassificationScanConfig.py`:

```
Config={
    "MaxEvents":int(3.e6),
    "NTestSamples":100000,
    "NClasses":4,

    "Epochs":1000,
    "BatchSize":1024,

...

    # Configure Running time callback
    # Set RunningTime to a value to stop training after N seconds.
    "RunningTime": 2*3600,

    # Load last trained version of this model configuration. (based on Name var below)
    "LoadPreviousModel":True
    }
```

These parameters are fixed and will be used by the Experiment to build the model. 

An important parameter in this configuration file is the `RunningTime`, which sets duration of the training. Using this parameter, you can train a model for a fix amount of time. You can rerun the job to continue training, which will automatically load the last successful training session, as set by `LoadPreviousModel` parameter.

[`CaloDNN/ClassificationScanConfig.py`](https://github.com/UTA-HEP-Computing/CaloDNN/blob/master/ClassificationScanConfig.py) is well commented. We suggest you read through the comments.

For hyper-parameter scanning, it would be cumbersome to generate a new configuration file for every
network we would like to try. Instead, **ScanConfig.py** uses a second dictionary  to specify
parameters that you would like to scan, and the **DLTools.Permutator** class to generate all possible
resulting configurations. For example the following lines:

```
# Parameters to scan and their scan points.
Params={    "optimizer":["'RMSprop'","'Adam'","'SGD'"],
            "Width":[32,64,128,256,512],
            "Depth":range(1,5),
            "lr":[0.01,0.001],
            "decay":[0.01,0.001],
          }
 ```

will generate 3 x 5 x 4 x 2 x 2 = 240 different configurations, which we can enumerate through. To check, we can
simply run the **ClassificationScanConfig.py** file:

In [1]:
%run -m CaloDNN.ClassificationScanConfig

HyperParameter Scan:  240 possible combiniations.
Combo[0]={'Width': 32, 'Depth': 1, 'lr': 0.01, 'optimizer': "'RMSprop'", 'decay': 0.01}
Combo[1]={'Width': 32, 'Depth': 1, 'lr': 0.01, 'optimizer': "'Adam'", 'decay': 0.01}
Combo[2]={'Width': 32, 'Depth': 1, 'lr': 0.01, 'optimizer': "'SGD'", 'decay': 0.01}
Combo[3]={'Width': 32, 'Depth': 1, 'lr': 0.001, 'optimizer': "'RMSprop'", 'decay': 0.01}
Combo[4]={'Width': 32, 'Depth': 1, 'lr': 0.001, 'optimizer': "'Adam'", 'decay': 0.01}
Combo[5]={'Width': 32, 'Depth': 1, 'lr': 0.001, 'optimizer': "'SGD'", 'decay': 0.01}
Combo[6]={'Width': 32, 'Depth': 1, 'lr': 0.01, 'optimizer': "'RMSprop'", 'decay': 0.001}
Combo[7]={'Width': 32, 'Depth': 1, 'lr': 0.01, 'optimizer': "'Adam'", 'decay': 0.001}
Combo[8]={'Width': 32, 'Depth': 1, 'lr': 0.01, 'optimizer': "'SGD'", 'decay': 0.001}
Combo[9]={'Width': 32, 'Depth': 1, 'lr': 0.001, 'optimizer': "'RMSprop'", 'decay': 0.001}
Combo[10]={'Width': 32, 'Depth': 1, 'lr': 0.001, 'optimizer': "'Adam'", 'decay': 0.

This should tell you the number possible configurations. We will select
specific ones using the **-s** flag when running the experiment.

In [ ]:
%run -im CaloDNN.ClassificationExperiment -- --Test -s 10

## Performing a Scan

From above, it should be appearant that in order you can easily try all possible configurations by running the same command with all possible values of the `-s` parameter. And since every configuration is independent, you can run the experiments in parallel. 

### PBS/Torque Batch System

On most GPU equipped clusters, like UTA-DL, a batch system allows you to submit "jobs" into "queues" which will then execute each job when appropriate resources become available. 

You can get a list of available queue, using the `qstat -Q` command:

In [1]:
!qstat -Q

Queue              Max   Tot   Ena   Str   Que   Run   Hld   Wat   Trn   Ext T         
----------------   ---   ---   ---   ---   ---   ---   ---   ---   ---   --- -         
gpu9thecount         1     0   yes   yes     0     0     0     0     0     0 E         
gpu5thecount         1     0   yes   yes     0     0     0     0     0     0 E         
gpu0oscar           20     1   yes   yes     0     1     0     0     0     0 E         
gpu0super            1     0   yes   yes     0     0     0     0     0     0 E         
gpu2thingtwo         1     0   yes   yes     0     0     0     0     0     0 E         
gpu1thecount         1     0   yes   yes     0     0     0     0     0     0 E         
gpu_queue            0     0   yes   yes     0     0     0     0     0     0 R         
cpu_queue            0     0   yes   yes     0     0     0     0     0     0 R         
gpu1thingone         1     0   yes   yes     0     0     0     0     0     0 E         
cpu_thingone        12     0   y

On the UTA-DL cluster, the queues are setup as follows. The `cpu_queue` and `gpu_queue` routing queues send jobs to CPU and GPU resources on each of 5 nodes:

    * thecount: 44-core 10 GPU
    * super: 24-core 4 GPU
    * thingone and thingtwo: 6 core 4 GPU each.
    * oscar: 6 core 2 GPU (used for Jupyter sessions).
    
Submitting to the queue system, requires you to write a script. For example, this is the script `CaloDNN/ScanJob.sh`:


This scripts creates a directory to store the `stdout/stderr` output of the job. Sets up the environment, and starts the job. To set the `-s` parameter, we use Torque's array job mechanism, which will set the `$PBS_ARRAYID` environment variable, to an interger as specified during submission.

So for example, to run configurations 10-20, we do (don't run this unless you mean it):

In [2]:
!qsub -q gpu_queue -t 10-20 CaloDNN/ScanJob.sh

7120.zoidberg


You can monitor your jobs using the `qstat` command:

In [5]:
!qstat

Job id                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
7119.zoidberg             STDIN            afarbin         00:00:01 R gpu0oscar      
7120-10.zoidberg          ScanJob.sh-10    afarbin         00:01:24 R gpu9thecount   
7120-11.zoidberg          ScanJob.sh-11    afarbin         00:02:44 R gpu8thecount   
7120-12.zoidberg          ScanJob.sh-12    afarbin         00:04:14 R gpu3super      
7120-13.zoidberg          ScanJob.sh-13    afarbin         00:03:24 R gpu1super      
7120-14.zoidberg          ScanJob.sh-14    afarbin         00:04:29 R gpu2super      
7120-15.zoidberg          ScanJob.sh-15    afarbin         00:04:31 R gpu0super      
7120-16.zoidberg          ScanJob.sh-16    afarbin         00:00:30 R gpu3thingtwo   
7120-17.zoidberg          ScanJob.sh-17    afarbin         00:00:03 R gpu2thingtwo   
7120-18.zoidberg          ScanJob.sh-18    afarbin         00:00:47 R gpu1

## Analysis

After you jobs start to complete, you can start viewing the performance using:

In [ ]:
!python -m DLAnalysis.Scan TrainedModels/

You can explore the performance of all of the models in your scan using the python notebook [`CaloDNN/AnalyzeScan-OptimizerStudy.ipynb`](https://github.com/UTA-HEP-Computing/CaloDNN/blob/master/AnalyzeScan-OptimizerStudy.ipynb). Simply make a copy of the notebook into your DLKit directory and navigate Jupyter to the notebook:

In [6]:
!cp CaloDNN/AnalyzeScan-OptimizerStudy.ipynb ./AnalyzeScan-MyStudy.ipynb

Similarly use can use [`CaloDNN/AnalyzerPerformance.ipynb`](https://github.com/UTA-HEP-Computing/CaloDNN/blob/master/AnalyzePerformance.ipynb) to study the performance of a specific model in detail.

## The Experiment

The main driver of the experiment, [`CaloDNN/ClassificationExperiment.py`](https://github.com/UTA-HEP-Computing/CaloDNN/blob/master/ClassificationExperiment.py), is well commented. In order for you to add you own models and modify things, you should carefully read through this file.